In [1]:
import pandas as pd
import geopandas as gpd
import pyproj
from tqdm import tqdm

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.subplots import make_subplots

import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
from folium.plugins import HeatMapWithTime

from branca.colormap import linear
# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

#### 기상상태별 교통사고 위험도

In [2]:
weather = pd.read_csv('open/external_open/도로교통공단_도로종류별 기상상태별 교통사고 통계_20221231.csv', encoding='cp949')
weather_ag = weather.drop(['도로종류'], axis=1).groupby('기상상태').sum()
weather_ag.index = ['기타', '눈', '맑음', '비', '안개', '흐림']

# 사고건수 대비 ECLO(인명피해 심각도) 계산 후 위험도 정규화(Min-Max) 진행
weather_ag['weather_danger'] = ((10 * weather_ag['사망자수']) + (5 * weather_ag['중상자수']) + (3 * weather_ag['경상자수']) + (1 * weather_ag['부상신고자수'])) / weather_ag['사고건수'] 
weather_ag['weather_danger'] = (weather_ag['weather_danger'] - weather_ag['weather_danger'].min())/ (weather_ag['weather_danger'].max() - weather_ag['weather_danger'].min())

# 날씨별 위험도 딕셔너리 생성
weather_danger_dic = dict(zip(weather_ag.index, weather_ag['weather_danger']))
weather_danger_dic

{'기타': 0.0,
 '눈': 0.630249016035219,
 '맑음': 0.14492007954600153,
 '비': 0.3198248805428553,
 '안개': 1.0,
 '흐림': 0.21384592769016494}

#### 도로형태별 교통사고 위험도

In [3]:
# 데이터로드, 마지막 두 행 선택 및 통일 후 재결합
roadshape2020_df = pd.read_csv('open/external_open/도로교통공단_도로형태별 교통사고(2020).csv', encoding='cp949')
roadshape2020_df.drop(8, inplace=True)
last_two_rows = roadshape2020_df.iloc[8:,:]
last_two_rows.loc[:, '도로형태_대분류'] = last_two_rows.iloc[0]['도로형태_대분류']
last_two_rows.loc[:, '도로형태'] = last_two_rows.iloc[0]['도로형태']
summed_rows = last_two_rows.groupby(['도로형태_대분류', '도로형태']).sum().reset_index()

roadshape2020_df = pd.concat([roadshape2020_df.iloc[:8,:], summed_rows])

# 데이터로드, 마지막 두 행 선택 및 통일 후 재결합
roadshape2021_df = pd.read_csv('open/external_open/도로교통공단_도로형태별 교통사고(2021).csv', encoding='cp949')
roadshape2021_df.drop(8, inplace=True)
last_two_rows = roadshape2021_df.iloc[8:,:]
last_two_rows.loc[:, '도로형태_대분류'] = last_two_rows.iloc[0]['도로형태_대분류']
last_two_rows.loc[:, '도로형태'] = last_two_rows.iloc[0]['도로형태']
summed_rows = last_two_rows.groupby(['도로형태_대분류', '도로형태']).sum().reset_index()

roadshape2021_df = pd.concat([roadshape2020_df.iloc[:8,:], summed_rows])

# 데이터로드, 마지막 행 편집
roadshape2022_df = pd.read_csv('open/external_open/도로교통공단_도로형태별 교통사고(2022).csv', encoding='cp949')
roadshape2022_df.drop(8, inplace=True)
roadshape2022_df.iloc[8:,:2] = ['기타', '기타']

roadshape_df = pd.concat([roadshape2020_df, roadshape2021_df, roadshape2022_df]).groupby(['도로형태_대분류', '도로형태']).sum().reset_index()
roadshape_df['도로형태'] = ['교차로안', '교차로부근', '교차로횡단보도내', '기타', '고가도로위', '교량', '기타', '지하차도(도로)내', '터널']
roadshape_df['roadshape_danger'] = ((10 * roadshape_df['사망자수']) + (5 * roadshape_df['중상자수']) + (3 * roadshape_df['경상자수']) + (1 * roadshape_df['부상신고자수'])) / roadshape_df['사고건수'] 
roadshape_df['roadshape_danger'] = (roadshape_df['roadshape_danger'] - roadshape_df['roadshape_danger'].min())/ (roadshape_df['roadshape_danger'].max() - roadshape_df['roadshape_danger'].min())
roadshape_df['도로형태'] = roadshape_df['도로형태_대분류'] + " - " + roadshape_df['도로형태']
roadshape_df = roadshape_df.iloc[:,1:]

# 날씨별 위험도 딕셔너리 생성
roadshape_danger_dic = dict(zip(roadshape_df['도로형태'], roadshape_df['roadshape_danger']))
roadshape_danger_dic['주차장 - 주차장'] = 0.5 # 없는 데이터 위험도 0.5로 통일
roadshape_danger_dic['미분류 - 미분류'] = 0.5 # 없는 데이터 위험도 0.5로 통일
roadshape_danger_dic

{'교차로 - 교차로안': 0.28371010316150946,
 '교차로 - 교차로부근': 0.21443340820597145,
 '교차로 - 교차로횡단보도내': 0.0,
 '기타 - 기타': 0.07573804112828718,
 '단일로 - 고가도로위': 0.4774196135018285,
 '단일로 - 교량': 0.48446609165016485,
 '단일로 - 기타': 0.2067045082897451,
 '단일로 - 지하차도(도로)내': 0.3439542378490483,
 '단일로 - 터널': 1.0,
 '주차장 - 주차장': 0.5,
 '미분류 - 미분류': 0.5}

#### 시간대별 교통사고 위험도

In [4]:
time2018_df = pd.read_csv('open/external_open/도로교통공단_도로형태별 시간대별 교통사고(2018).csv', encoding='cp949')
time2018_df = time2018_df.iloc[:,2:]
time2018_df = time2018_df.groupby(['발생시간_분류']).sum().reset_index()
time2018_df.rename(columns={'발생시간_분류':'시간대'}, inplace=True)

time2019_df = pd.read_csv('open/external_open/도로교통공단_도로형태별 시간대별 교통사고(2019).csv', encoding='cp949')
time2019_df = time2019_df.iloc[:,2:]
time2019_df = time2019_df.groupby(['발생시간_분류']).sum().reset_index()
time2019_df.rename(columns={'발생시간_분류':'시간대'}, inplace=True)

time2021_df = pd.read_csv('open/external_open/도로교통공단_도로형태별 시간대별 교통사고(2021).csv', encoding='cp949')
time2021_df = time2021_df.iloc[:,2:]
time2021_df = time2021_df.groupby(['시간대']).sum().reset_index()

time2022_df = pd.read_csv('open/external_open/도로교통공단_도로형태별 시간대별 교통사고(2022).csv', encoding='cp949')
time2022_df = time2022_df.iloc[:,2:]
time2022_df = time2022_df.groupby(['시간대']).sum().reset_index()

time_df = pd.concat([time2018_df, time2019_df, time2021_df, time2022_df]).groupby(['시간대']).sum().reset_index()
time_df['time_danger'] = ((10 * time_df['사망자수']) + (5 * time_df['중상자수']) + (3 * time_df['경상자수']) + (1 * time_df['부상신고자수'])) / time_df['사고건수'] 
time_df['time_danger'] = (time_df['time_danger'] - time_df['time_danger'].min())/ (time_df['time_danger'].max() - time_df['time_danger'].min())

time_df_danger_dic = {}
count = 0
for f in range(3):
    for b in range(10):
        if f"{f}{b}" == "24":
            break
        time_df_danger_dic[f"{f}{b}"] = time_df['time_danger'][count//2]
        count += 1

time_df_danger_dic

{'00': 0.9155015885837771,
 '01': 0.9155015885837771,
 '02': 0.9576499818930245,
 '03': 0.9576499818930245,
 '04': 1.0,
 '05': 1.0,
 '06': 0.3539509636954647,
 '07': 0.3539509636954647,
 '08': 0.08551203039087009,
 '09': 0.08551203039087009,
 '10': 0.4064413368531681,
 '11': 0.4064413368531681,
 '12': 0.46508264354822065,
 '13': 0.46508264354822065,
 '14': 0.4517063261223543,
 '15': 0.4517063261223543,
 '16': 0.20146548927365882,
 '17': 0.20146548927365882,
 '18': 0.0,
 '19': 0.0,
 '20': 0.23493920339978955,
 '21': 0.23493920339978955,
 '22': 0.46681555165169286,
 '23': 0.46681555165169286}

#### 사고유형별 교통사고 위험도

In [5]:
case2020_df = pd.read_csv('open/external_open/도로교통공단_사고유형별 교통사고(2020).csv', encoding='cp949')
case2020_df = case2020_df.iloc[:, [0,3,4,5,6,7]]
case2020_df = case2020_df.groupby(['사고유형대분류']).sum().reset_index()
case2020_df.rename(columns={'사고유형대분류':'사고유형'}, inplace=True)
case2020_df = case2020_df.iloc[:3]

case2021_df = pd.read_csv('open/external_open/도로교통공단_사고유형별 교통사고(2021).csv', encoding='cp949')
case2021_df = case2021_df.iloc[:, [0,3,4,5,6,7]]
case2021_df = case2021_df.groupby(['사고유형대분류']).sum().reset_index()
case2021_df.rename(columns={'사고유형대분류':'사고유형'}, inplace=True)
case2021_df = case2021_df.iloc[:3]

case2022_df = pd.read_csv('open/external_open/도로교통공단_사고유형별 교통사고(2022).csv', encoding='cp949')
case2022_df = case2022_df.iloc[:, [0,3,4,5,6,7]]
case2022_df = case2022_df.groupby(['사고유형대분류']).sum().reset_index()
case2022_df.rename(columns={'사고유형대분류':'사고유형'}, inplace=True)
case2022_df = case2022_df.iloc[:3]

case_df = pd.concat([case2020_df, case2021_df, case2022_df]).groupby(['사고유형']).sum().reset_index()
case_df['case_danger'] = ((10 * case_df['사망자수']) + (5 * case_df['중상자수']) + (3 * case_df['경상자수']) + (1 * case_df['부상신고자수'])) / case_df['사고건수'] 
case_df['case_danger'] = (case_df['case_danger'] - case_df['case_danger'].min())/ (case_df['case_danger'].max() - case_df['case_danger'].min())

case_danger_dic = dict(zip(case_df['사고유형'], case_df['case_danger']))
case_danger_dic

{'차대사람': 0.0, '차대차': 1.0, '차량단독': 0.5945617604734095}

#### 요일별 위험도

In [6]:
day2018_df = pd.read_csv('open/external_open/도로교통공단_요일별 시간대별 교통사고(2018).csv', encoding='cp949')
day2018_df.drop(['Unnamed: 7'], axis=1, inplace=True)
day2018_df.dropna(inplace=True)
day2018_df = day2018_df.iloc[:, [0,2,3,4,5,6]]
day2018_df = day2018_df.groupby(['요일']).sum().reset_index()

day2019_df = pd.read_csv('open/external_open/도로교통공단_요일별 시간대별 교통사고(2019).csv', encoding='cp949')
day2019_df.drop(['Unnamed: 7'], axis=1, inplace=True)
day2019_df.dropna(inplace=True)
day2019_df = day2019_df.iloc[:, [0,2,3,4,5,6]]
day2019_df = day2019_df.groupby(['요일']).sum().reset_index()

day2020_df = pd.read_csv('open/external_open/도로교통공단_요일별 시간대별 교통사고(2020).csv', encoding='cp949')
day2020_df = day2020_df.iloc[:, [0,2,3,4,5,6]]
day2020_df = day2020_df.groupby(['요일']).sum().reset_index()

day2021_df = pd.read_csv('open/external_open/도로교통공단_요일별 시간대별 교통사고(2021).csv', encoding='cp949')
day2021_df = day2021_df.iloc[:, [0,2,3,4,5,6]]
day2021_df = day2021_df.groupby(['요일']).sum().reset_index()

day2022_df = pd.read_csv('open/external_open/도로교통공단_요일별 시간대별 교통사고(2022).csv', encoding='cp949')
day2022_df = day2022_df.iloc[:, [0,2,3,4,5,6]]
day2022_df = day2022_df.groupby(['요일']).sum().reset_index()

day_df = pd.concat([day2018_df, day2019_df, day2020_df, day2021_df, day2022_df]).groupby(['요일']).sum().reset_index()
day_df['day_danger'] = ((10 * day_df['사망자수']) + (5 * day_df['중상자수']) + (3 * day_df['경상자수']) + (1 * day_df['부상신고자수'])) / day_df['사고건수'] 
day_df['day_danger'] = (day_df['day_danger'] - day_df['day_danger'].min())/ (day_df['day_danger'].max() - day_df['day_danger'].min())

day_danger_dic = dict(zip(day_df['요일'], day_df['day_danger']))
day_danger_dic

{'금': 0.06940624709800688,
 '목': 0.0,
 '수': 0.021974592426828834,
 '월': 0.07538748611096166,
 '일': 1.0,
 '토': 0.725915673064352,
 '화': 0.03275592362861103}

#### 노면상태별 위험도

In [19]:
roadsurface_df = pd.read_csv('open/external_open/노면상태별 교통사고.csv')
roadsurface_df['roadsurface_danger'] = (roadsurface_df['사망자수'] + roadsurface_df['부상자수']) / roadsurface_df['사고건수'] 
roadsurface_df['roadsurface_danger'] = (roadsurface_df['roadsurface_danger'] - roadsurface_df['roadsurface_danger'].min())/ (roadsurface_df['roadsurface_danger'].max() - roadsurface_df['roadsurface_danger'].min())

roadsurface_danger_dic = dict(zip(roadsurface_df['상태'], roadsurface_df['roadsurface_danger']))
roadsurface_danger_dic

{'건조': 0.22710326004512565,
 '젖음/습기': 0.2631478069391325,
 '서리/결빙': 1.0,
 '적설': 0.8530809155632956,
 '기타': 0.09019011664881801,
 '침수': 0.0}

#### 월별 위험도

In [30]:
month2019_df = pd.read_csv('open/external_open/도로교통공단_월별 요일별 교통사고(2019).csv', encoding='cp949')
month2019_df = month2019_df.iloc[:, [0,2,3,4,5,6]]
month2019_df.rename(columns={'월':'발생월'}, inplace=True)
month2019_df = month2019_df.groupby(['발생월']).sum().reset_index()

month2020_df = pd.read_csv('open/external_open/도로교통공단_월별 요일별 교통사고(2020).csv', encoding='cp949')
month2020_df = month2020_df.iloc[:, [0,2,3,4,5,6]]
month2020_df = month2020_df.groupby(['발생월']).sum().reset_index()

month2021_df = pd.read_csv('open/external_open/도로교통공단_월별 요일별 교통사고(2021).csv', encoding='cp949')
month2021_df = month2021_df.iloc[:, [0,2,3,4,5,6]]
month2021_df = month2021_df.groupby(['발생월']).sum().reset_index()

month2022_df = pd.read_csv('open/external_open/도로교통공단_월별 요일별 교통사고(2022).csv', encoding='cp949')
month2022_df = month2022_df.iloc[:, [0,2,3,4,5,6]]
month2022_df = month2022_df.groupby(['발생월']).sum().reset_index()

month_df = pd.concat([month2019_df, month2020_df, month2021_df, month2022_df]).groupby(['발생월']).sum().reset_index()
month_df['month_danger'] = ((10 * month_df['사망자수']) + (5 * month_df['중상자수']) + (3 * month_df['경상자수']) + (1 * month_df['부상신고자수'])) / month_df['사고건수'] 
month_df['month_danger'] = (month_df['month_danger'] - month_df['month_danger'].min())/ (month_df['month_danger'].max() - month_df['month_danger'].min())

month_danger_dic = dict(zip(month_df['발생월'], month_df['month_danger']))
month_danger_dic

{1: 0.8247558671953833,
 2: 0.7728967332121681,
 3: 0.5153471264144931,
 4: 0.8323063848167668,
 5: 0.7231366243147488,
 6: 0.6083855665889639,
 7: 0.5120610426494504,
 8: 1.0,
 9: 0.36311241627115187,
 10: 0.7449907104629871,
 11: 0.25470992254305075,
 12: 0.0}

#### trainset 전처리

In [78]:
train_set = pd.read_csv('open/train.csv')
train_set = pd.concat([train_set.iloc[:,:8], train_set.iloc[:,-1:]], axis=1)
train_set['요일'] = train_set['요일'].apply(lambda x : x[0])
train_set['시간대'] = train_set['사고일시'].apply(lambda x : x[-2:])
train_set['발생월'] = train_set['사고일시'].apply(lambda x : int(x[5:7]))
train_set = train_set.iloc[:, [0,10,9,2,3,4,5,6,7,8]]
train_set

,ID,발생월,시간대,요일,기상상태,시군구,도로형태,노면상태,사고유형,ECLO
0,ACCIDENT_00000,1,00,화,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,5
1,ACCIDENT_00001,1,00,화,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,3
2,ACCIDENT_00002,1,01,화,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,3
3,ACCIDENT_00003,1,02,화,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,5
4,ACCIDENT_00004,1,04,화,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,3
...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,12,19,금,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,3
39605,ACCIDENT_39605,12,19,금,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,3
39606,ACCIDENT_39606,12,21,금,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,10
39607,ACCIDENT_39607,12,22,금,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,3


In [79]:
# 지역구별 ECLO를 이용해서 정규화
district_df = train_set[['시군구', 'ECLO']].groupby(['시군구']).mean().reset_index()
district_df['ECLO'] = (district_df['ECLO'] - district_df['ECLO'].min())/ (district_df['ECLO'].max() - district_df['ECLO'].min())
district_danger_dic = dict(zip(district_df['시군구'], district_df['ECLO']))
district_danger_dic

{'대구광역시 남구 대명동': 0.2698684962835906,
 '대구광역시 남구 봉덕동': 0.3293333333333333,
 '대구광역시 남구 이천동': 0.29617117117117114,
 '대구광역시 달서구 갈산동': 0.32196969696969696,
 '대구광역시 달서구 감삼동': 0.3007005899705015,
 '대구광역시 달서구 대곡동': 0.3098404255319149,
 '대구광역시 달서구 대천동': 0.3168154761904762,
 '대구광역시 달서구 도원동': 0.28969109195402293,
 '대구광역시 달서구 두류동': 0.274238782051282,
 '대구광역시 달서구 본동': 0.3114224137931035,
 '대구광역시 달서구 본리동': 0.266401792991035,
 '대구광역시 달서구 상인동': 0.24507874015748027,
 '대구광역시 달서구 성당동': 0.2744605654761904,
 '대구광역시 달서구 송현동': 0.26443690916563906,
 '대구광역시 달서구 신당동': 0.2766404199475065,
 '대구광역시 달서구 용산동': 0.31415822479498307,
 '대구광역시 달서구 월성동': 0.28944174757281554,
 '대구광역시 달서구 월암동': 0.3899026763990267,
 '대구광역시 달서구 유천동': 0.29188255613126074,
 '대구광역시 달서구 이곡동': 0.27588483888008447,
 '대구광역시 달서구 장기동': 0.30502322880371663,
 '대구광역시 달서구 장동': 0.2923333333333333,
 '대구광역시 달서구 죽전동': 0.3649217809867629,
 '대구광역시 달서구 진천동': 0.24320282762370854,
 '대구광역시 달서구 파호동': 0.3990384615384615,
 '대구광역시 달서구 호림동': 0.43749999999999994,
 '대구광역시

In [80]:
train_set['발생월'] = train_set['발생월'].map(month_danger_dic)
train_set['시간대'] = train_set['시간대'].map(time_df_danger_dic)
train_set['요일'] = train_set['요일'].map(day_danger_dic)
train_set['기상상태'] = train_set['기상상태'].map(weather_danger_dic)
train_set['시군구'] = train_set['시군구'].map(district_danger_dic)
train_set['도로형태'] = train_set['도로형태'].map(roadshape_danger_dic)
train_set['노면상태'] = train_set['노면상태'].map(roadsurface_danger_dic)
train_set['사고유형'] = train_set['사고유형'].map(case_danger_dic)
train_set

,ID,발생월,시간대,요일,기상상태,시군구,도로형태,노면상태,사고유형,ECLO
0,ACCIDENT_00000,0.824756,0.915502,0.032756,0.144920,0.243639,0.206705,0.227103,0.0,5
1,ACCIDENT_00001,0.824756,0.915502,0.032756,0.213846,0.300701,0.206705,0.227103,0.0,3
2,ACCIDENT_00002,0.824756,0.915502,0.032756,0.144920,0.313673,0.206705,0.227103,0.0,3
3,ACCIDENT_00003,0.824756,0.957650,0.032756,0.144920,0.234448,0.206705,0.227103,1.0,5
4,ACCIDENT_00004,0.824756,1.000000,0.032756,0.144920,0.276970,0.206705,0.227103,1.0,3
...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,0.000000,0.000000,0.069406,0.144920,0.322516,0.283710,0.227103,1.0,3
39605,ACCIDENT_39605,0.000000,0.000000,0.069406,0.144920,0.245079,0.206705,0.227103,1.0,3
39606,ACCIDENT_39606,0.000000,0.234939,0.069406,0.144920,0.289442,0.283710,0.227103,1.0,10
39607,ACCIDENT_39607,0.000000,0.466816,0.069406,0.144920,0.292333,0.075738,0.227103,1.0,3


#### testset 전처리

In [82]:
test_set = pd.read_csv('open/test.csv')
test_set['요일'] = test_set['요일'].apply(lambda x : x[0])
test_set['시간대'] = test_set['사고일시'].apply(lambda x : x[-2:])
test_set['발생월'] = test_set['사고일시'].apply(lambda x : int(x[5:7]))
test_set = test_set.iloc[:, [0,9,8,2,3,4,5,6,7]]
test_set

,ID,발생월,시간대,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,ACCIDENT_39609,1,01,토,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,ACCIDENT_39610,1,01,토,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,ACCIDENT_39611,1,04,토,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차
3,ACCIDENT_39612,1,04,토,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차
4,ACCIDENT_39613,1,06,토,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차
...,...,...,...,...,...,...,...,...,...
10958,ACCIDENT_50567,12,18,토,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
10959,ACCIDENT_50568,12,18,토,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
10960,ACCIDENT_50569,12,20,토,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
10961,ACCIDENT_50570,12,20,토,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차


In [83]:
test_set['발생월'] = test_set['발생월'].map(month_danger_dic)
test_set['시간대'] = test_set['시간대'].map(time_df_danger_dic)
test_set['요일'] = test_set['요일'].map(day_danger_dic)
test_set['기상상태'] = test_set['기상상태'].map(weather_danger_dic)
test_set['시군구'] = test_set['시군구'].map(district_danger_dic)
test_set['도로형태'] = test_set['도로형태'].map(roadshape_danger_dic)
test_set['노면상태'] = test_set['노면상태'].map(roadsurface_danger_dic)
test_set['사고유형'] = test_set['사고유형'].map(case_danger_dic)
test_set

,ID,발생월,시간대,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,ACCIDENT_39609,0.824756,0.915502,0.725916,0.14492,0.318540,0.283710,0.227103,0.0
1,ACCIDENT_39610,0.824756,0.915502,0.725916,0.14492,0.278709,0.206705,0.227103,0.0
2,ACCIDENT_39611,0.824756,1.000000,0.725916,0.14492,0.326531,0.283710,0.227103,1.0
3,ACCIDENT_39612,0.824756,1.000000,0.725916,0.14492,0.263105,0.206705,0.227103,1.0
4,ACCIDENT_39613,0.824756,0.353951,0.725916,0.14492,0.300701,0.283710,0.227103,1.0
...,...,...,...,...,...,...,...,...,...
10958,ACCIDENT_50567,0.000000,0.000000,0.725916,0.14492,0.269868,1.000000,0.227103,1.0
10959,ACCIDENT_50568,0.000000,0.000000,0.725916,0.14492,0.324351,0.206705,0.227103,1.0
10960,ACCIDENT_50569,0.000000,0.234939,0.725916,0.14492,0.377726,0.206705,0.227103,1.0
10961,ACCIDENT_50570,0.000000,0.234939,0.725916,0.14492,0.343271,0.214433,0.227103,1.0


In [ ]:
security_light = pd.read_csv('open/external_open/대구 보안등 정보.csv', encoding='cp949')
security_light

C:\Users\user\AppData\Local\Temp\ipykernel_26440\2934138998.py:1: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



,보안등위치명,설치개수,소재지도로명주소,소재지지번주소,위도,경도,설치연도,설치형태
0,대명1동1,1,대구광역시 남구 현충로 155,대구광역시 남구 대명동 1722-4,35.846703,128.579803,2016.0,한전주
1,대명1동2,1,대구광역시 남구 현충로31길 10-1,대구광역시 남구 대명동 1723-11,35.846863,128.579406,2016.0,한전주
2,대명1동3,1,대구광역시 남구 현충로31길 9-8,대구광역시 남구 대명동 1722-56,35.846341,128.579342,2017.0,건축물
3,대명1동4,2,대구광역시 남구 현충로31길 19-11,대구광역시 남구 대명동 1722-41,35.846368,128.578922,2016.0,한전주
4,대명1동5,1,대구광역시 남구 현충로29길 22-2,대구광역시 남구 대명동 1721-22,35.845995,128.578858,2016.0,한전주
...,...,...,...,...,...,...,...,...
71908,대봉2동 165-9,1,NaN,대구광역시 중구 대봉동 165-9,35.857556,128.600023,NaN,한전주
71909,대봉2동 88-85,1,NaN,대구광역시 중구 대봉동 88-85,35.859205,128.600046,NaN,한전주
71910,대봉2동 719-20,1,NaN,대구광역시 중구 대봉동 719-20,35.859442,128.599627,NaN,한전주
71911,대봉2동 738-13,1,NaN,대구광역시 중구 대봉동 738-13,35.858244,128.597614,2010.0,건축물


In [ ]:
kid_zone = pd.read_csv('open/external_open/대구 어린이 보호 구역 정보.csv', encoding='cp949')
kid_zone

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자
0,초등학교,남도초등학교,대구광역시 남구 현충동길 74(대명동),대구광역시 남구 대명동 1709,35.845027,128.581402,대구광역시,남부경찰서,Y,7.0,6~8,2020-03-23
1,초등학교,영선초등학교,대구광역시 남구 영선길96(이천동),대구광역시 남구 이천동 477,35.852901,128.596014,대구광역시,남부경찰서,Y,8.0,6~10,2020-03-23
2,초등학교,성명초등학교,대구광역시 남구 성당로 30길 55(대명동),대구광역시 남구 대명동 3050,35.845152,128.570825,대구광역시,남부경찰서,Y,14.0,8~12,2020-03-23
3,초등학교,남덕초등학교,대구광역시 남구 앞산순환로 93길 33,대구광역시 남구 대명동 531-1,35.833042,128.573949,대구광역시,남부경찰서,Y,6.0,6~8,2020-03-23
4,초등학교,대명초등학교,대구광역시 남구 대명로 110,대구광역시 남구 대명동 960,35.838869,128.568889,대구광역시,남부경찰서,Y,5.0,6~10,2020-03-23
...,...,...,...,...,...,...,...,...,...,...,...,...
635,어린이집,남산교회어린이집,대구광역시 중구 관덕정길 16,대구광역시 중구 남산동 941-1,35.865040,128.590565,대구광역시 중구청,대구광역시 중부경찰서,Y,5.0,6~10,2019-06-01
636,초등학교,대구초등학교,대구광역시 중구 명륜로23길 80,대구광역시 중구 봉산동 57,35.864037,128.595370,대구광역시 중구청,대구광역시 중부경찰서,Y,12.0,3~12,2019-06-01
637,어린이집,대구삼성어린이집,대구광역시 중구 달구벌대로 2016-40,대구광역시 중구 남산동 153-2,35.864469,128.584301,대구광역시 중구청,대구광역시 중부경찰서,Y,2.0,7~8,2019-06-01
638,어린이집,동화나라어린이집,대구광역시 중구 남산로3안길 20,대구광역시 중구 남산동 2623-6,35.860371,128.577492,대구광역시 중구청,대구광역시 중부경찰서,Y,5.0,6~8,2019-06-01


In [ ]:
# data1 = pd.read_csv('open/external_open/countrywide_accident_1.csv')
# data2 = pd.read_csv('open/external_open/countrywide_accident_2.csv')
# data3 = pd.read_csv('open/external_open/countrywide_accident_3.csv')
# data4 = pd.read_csv('open/external_open/countrywide_accident_4.csv')
# data = pd.concat([data1, data2, data3, data4]).reset_index(drop=True)
# data.to_csv('open/external_open/countrywide_accident.csv', index=False)

#### 대구광역시 격자데이터

In [ ]:
grid_500 = gpd.read_file('open/external_open/대구 빅데이터 마트 데이터/0. Base/대구광역시_500.gpkg')
grid_500 = grid_500.to_crs(epsg=4326)
grid_500

,id,left,top,right,bottom,geometry
0,69,1.077033e+06,1.746587e+06,1.077533e+06,1.746087e+06,"MULTIPOLYGON (((128.35675 35.70806, 128.35710 ..."
1,70,1.077033e+06,1.746087e+06,1.077533e+06,1.745587e+06,"MULTIPOLYGON (((128.35180 35.70402, 128.35230 ..."
2,71,1.077033e+06,1.745587e+06,1.077533e+06,1.745087e+06,"MULTIPOLYGON (((128.35148 35.69943, 128.35148 ..."
3,72,1.077033e+06,1.745087e+06,1.077533e+06,1.744587e+06,"MULTIPOLYGON (((128.35205 35.69522, 128.35192 ..."
4,73,1.077033e+06,1.744587e+06,1.077533e+06,1.744087e+06,"MULTIPOLYGON (((128.35344 35.69057, 128.35330 ..."
...,...,...,...,...,...,...
3771,6746,1.113533e+06,1.766087e+06,1.114033e+06,1.765587e+06,"MULTIPOLYGON (((128.76093 35.88447, 128.76100 ..."
3772,6747,1.113533e+06,1.765587e+06,1.114033e+06,1.765087e+06,"MULTIPOLYGON (((128.75909 35.88007, 128.75897 ..."
3773,6748,1.113533e+06,1.765087e+06,1.114033e+06,1.764587e+06,"MULTIPOLYGON (((128.75953 35.87574, 128.75962 ..."
3774,6749,1.113533e+06,1.764587e+06,1.114033e+06,1.764087e+06,"MULTIPOLYGON (((128.75939 35.87123, 128.75933 ..."


In [ ]:
# map 생성
m = folium.Map(location=[35.87285921915183, 128.5969619227697], zoom_start=12)

# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in grid_500.iterrows():
    popup_text = f"{row['geometry']}"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color='black': {'color': 'black', 'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=100))

# 맵 출력
m

#### 대구광역시 수치표고

In [ ]:
digital_elevation_500 = gpd.read_file('open/external_open/대구 빅데이터 마트 데이터/1. 환경/8. 수치표고/수치표고_500.gpkg')
digital_elevation_500 = digital_elevation_500.to_crs(epsg=4326)
#digital_elevation_500['NUMPOINTS'] = digital_elevation_500['NUMPOINTS'].astype(int)
digital_elevation_500

In [ ]:
# map 생성
m = folium.Map(location=[35.87285921915183, 128.5969619227697], zoom_start=12)

folium.TileLayer('openstreetmap', overlay=False).add_to(m)
m_elevation = folium.FeatureGroup(name="수치표고", overlay=False)
# GeoDataFrame을 순회하면서 Polygon을 지도에 추가
for idx, row in digital_elevation_500.iterrows():
    popup_text = f"{row['DN_mean']}"
    
    # 그라데이션을 위한 colormap 정의
    colormap = linear.YlOrRd_09.scale(4.25, 1062.83871)
    
    # fill color를 그라데이션으로 지정
    fill_color = colormap(row['DN_mean'])
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=fill_color: {'fillColor': color, 'color': 'black', 'weight': 0.5}
    ).add_to(m_elevation).add_child(folium.Popup(popup_text, max_width=100))

# colormap을 맵에 추가 (선택사항)
colormap.add_to(m)

m_elevation.add_to(m)

# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)
# 맵 출력
m